In [1]:
from typing import Dict, List

In [2]:
folders = glob("grant_pdf_17900731_18641101/00000001-X009741H/00/*/*")

In [4]:
def prepare_text_from_uspto(folder):
    start = 0
    xml = glob(folder + "/*.xml")[0]
    for r in read_str(xml):
        if "<claims-pages>" in r:
            start = 1
        if "</claims-pages>" in r:
            break
        if start:
            if "begin" in r:
                begin = int(re.findall("\d+", r)[0])
            if "end" in r:
                end = int(re.findall("\d+", r)[0])
    claim_pages = range(begin - 1, end)
    for page in claim_pages:
        for f in glob(folder + "/*.txt"):
            if f.endswith(f"{page}.txt"):
                files.append(f)

    data = []
    for f in files:
        data.extend(read_str(f))

In [2]:
def parse(data) -> List:
    num = dict()
    i = 1
    for r in data:
        if "witness" in r.lower() and num:
            break
        if r.startswith(f"{i}. "):
            num[i] = list()
            num[i].append(r)
            i += 1
        else:
            if num:
                num[i - 1].append(r)

    if num:  # if we do find claims by numberings
        return [(folder, k, " ".join(v)) for k, v in num.items()]
    else:
        start = 0
        claim_rows = []
        for i, r in enumerate(data):
            if "claim" in r or "desire to" in r or "secure by" in r or "is-" in r:
                claim_rows.append(i)
        res = []
        if len(claim_rows) == 0:
            return {}
        for r in data[claim_rows[-1] + 1 :]:
            if not r.strip() and start == 0:
                start = 1
                continue
            if not r.strip() and start == 1:
                break
            if start:
                res.append(r)
        return [(folder, 1, "1. " + " ".join(res))]

NameError: name 'List' is not defined